In [1]:
import os
import pickle
import re
from pathlib import Path
import pandas as pd
from tsfresh.feature_selection.relevance import calculate_relevance_table


In [2]:
# Load features
ext_dwell = pd.read_excel(
    "S02_af_features.xlsx", sheet_name="af_dwell", index_col="sample_no"
)
ext_weld = pd.read_excel(
    "S02_af_features.xlsx", sheet_name="af_weld", index_col="sample_no"
)

In [3]:
ext_dwell

,Fz__variance_larger_than_standard_deviation,Fz__has_duplicate_max,Fz__has_duplicate_min,Fz__has_duplicate,Fz__sum_values,Fz__abs_energy,Fz__mean_abs_change,Fz__mean_change,Fz__mean_second_derivative_central,Fz__median,...,Fz__fourier_entropy__bins_3,Fz__fourier_entropy__bins_5,Fz__fourier_entropy__bins_10,Fz__fourier_entropy__bins_100,Fz__permutation_entropy__dimension_3__tau_1,Fz__permutation_entropy__dimension_4__tau_1,Fz__permutation_entropy__dimension_5__tau_1,Fz__permutation_entropy__dimension_6__tau_1,Fz__permutation_entropy__dimension_7__tau_1,Fz__mean_n_absolute_max__number_of_maxima_7
sample_no,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,2292.314200,5263.882785,0.067820,-0.000466,-0.000017,2.276870,...,0.090729,0.136002,0.181214,0.380783,1.692953,2.579183,3.470798,4.453641,5.302442,2.840246
2,0,0,0,1,3925.106890,10278.990476,0.081735,-0.000060,0.000039,2.611690,...,0.090729,0.090729,0.136002,0.226363,1.706271,2.499941,3.162247,3.848210,4.512462,3.025519
3,0,0,0,1,5147.686690,13260.798765,0.079072,-0.000198,0.000031,2.566480,...,0.090729,0.090729,0.136002,0.226363,1.707461,2.513931,3.306369,4.182854,5.033741,3.008877
4,0,0,0,1,2904.696910,8438.959197,0.079856,-0.000459,0.000037,2.901610,...,0.090729,0.090729,0.136002,0.226363,1.707924,2.496673,3.169187,3.902923,4.588635,3.244554
5,0,0,0,1,1711.214419,2098.143943,0.066067,-0.000819,-0.000048,1.022730,...,0.090729,0.170467,0.280179,0.765101,1.772258,3.078485,4.446464,5.776713,6.610237,2.174843
6,0,0,0,1,3325.888910,5713.061947,0.111352,-0.000574,0.000087,1.720100,...,0.125256,0.136002,0.181214,0.316475,1.706424,2.501311,3.139175,3.810751,4.454771,2.610774
7,0,0,0,1,2162.477250,4845.482539,0.115855,-0.001713,0.000046,2.054840,...,0.125256,0.136002,0.181214,0.361435,1.703976,2.465077,3.078517,3.729684,4.365196,3.354126
8,0,0,0,1,2261.881960,3440.071923,0.056214,-0.000507,0.000067,1.473560,...,0.125256,0.136002,0.181214,0.350689,1.698840,2.526089,3.250819,4.048937,4.833185,2.191946
9,0,0,0,1,4270.616530,9228.344912,0.078709,-0.000700,0.000050,2.121630,...,0.125256,0.125256,0.215617,0.395585,1.706441,2.533300,3.236346,4.001534,4.740505,3.428850


In [4]:
ext_weld

,Fz__variance_larger_than_standard_deviation,Fz__has_duplicate_max,Fz__has_duplicate_min,Fz__has_duplicate,Fz__sum_values,Fz__abs_energy,Fz__mean_abs_change,Fz__mean_change,Fz__mean_second_derivative_central,Fz__median,...,Fz__fourier_entropy__bins_3,Fz__fourier_entropy__bins_5,Fz__fourier_entropy__bins_10,Fz__fourier_entropy__bins_100,Fz__permutation_entropy__dimension_3__tau_1,Fz__permutation_entropy__dimension_4__tau_1,Fz__permutation_entropy__dimension_5__tau_1,Fz__permutation_entropy__dimension_6__tau_1,Fz__permutation_entropy__dimension_7__tau_1,Fz__mean_n_absolute_max__number_of_maxima_7
sample_no,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,23410.069643,43338.823860,0.056338,-0.000105,-1.152566e-05,1.891870,...,0.090729,0.200814,0.280179,0.732018,1.778420,3.085498,4.331367,5.627806,6.842932,2.304806
2,0,0,0,1,22352.769318,39351.742522,0.054584,-0.000171,-2.178629e-07,1.776340,...,0.125256,0.136002,0.273489,0.630837,1.767604,3.051091,4.258642,5.523756,6.692855,3.005471
3,0,0,1,1,23248.980550,42280.129982,0.061530,-0.000124,-1.175190e-05,1.847580,...,0.125256,0.200814,0.294982,0.631349,1.777078,3.071510,4.306132,5.564160,6.660217,2.586053
4,0,0,0,1,20550.524463,38444.388684,0.052436,-0.000189,-1.455964e-05,1.846335,...,0.125256,0.170467,0.260704,0.705343,1.745844,2.867287,4.114612,5.401085,6.537980,2.914564
5,0,0,0,1,15253.195715,21365.276071,0.053669,-0.000044,-1.587237e-05,1.443480,...,0.125256,0.200814,0.294982,0.721454,1.770052,3.051995,4.323799,5.525498,6.573618,1.750769
6,0,0,0,1,13874.984210,17690.434465,0.060816,-0.000100,-1.408576e-05,1.288145,...,0.090729,0.200814,0.280179,0.620602,1.768802,3.036285,4.293350,5.453949,6.442128,1.583490
7,0,0,0,1,16000.384610,26736.136770,0.052329,-0.000132,-6.397118e-06,1.672060,...,0.125256,0.200814,0.352599,0.856815,1.751144,2.947722,4.261066,5.580069,6.722255,1.992956
8,0,0,0,1,13335.651239,18777.831219,0.054852,-0.000122,-9.659657e-06,1.392470,...,0.090729,0.090729,0.278098,0.642095,1.759027,2.999614,4.267891,5.504867,6.513980,1.797624
9,0,0,0,1,15204.525566,24185.564339,0.060926,-0.000157,-1.627069e-05,1.587870,...,0.125256,0.299054,0.363346,0.749637,1.762795,3.033120,4.335562,5.559250,6.572327,1.947539


In [5]:
# Load stress data
stresses = pd.read_excel("S01_residual_stress_wide_format.xlsx")
stresses = stresses.set_index("sample_no")  # Set sample_no as index for tsfresh
stresses

,R,W,D,AA5052_L1,AA5052_L2,AA5052_L3,AA5052_L4,AA5052_L5,AA5052_L6,AA5052_L7,...,Center_L1,Center_L2,Center_L3,Center_L4,Center_L5,Center_L6,Center_L7,AA5052_mean,AA6061_mean,Center_mean
sample_no,,,,,,,,,,,,,,,,,,,,,
1,1400,60,10,28,12.000000,33.000000,42,23.000000,10,18.000000,...,12,26.000000,29,24,19.000000,6,6,23.714286,16.293956,17.428571
2,1400,60,15,14,10.000000,26.000000,27,21.000000,-4,-10.000000,...,17,25.000000,23,14,10.000000,3,2,12.000000,-28.428571,13.428571
3,1400,60,20,10,14.000000,14.000000,25,11.000000,3,6.000000,...,12,28.000000,21,22,12.000000,16,8,11.857143,-8.428571,17.000000
4,1400,70,10,10,36.000000,22.000000,11,14.000000,23,8.000000,...,20,24.000000,29,18,11.000000,18,3,17.714286,8.857143,17.571429
5,1400,70,15,6,14.000000,8.000000,7,13.000000,12,10.000000,...,14,4.000000,8,11,9.000000,4,6,10.000000,26.000000,8.000000
6,1400,70,20,15,11.000000,-13.000000,-1,10.000000,-20,0.000000,...,11,12.000000,13,4,7.000000,2,12,0.285714,-38.142857,8.714286
7,1400,80,10,63,45.000000,24.000000,31,19.000000,46,2.000000,...,12,15.000000,17,20,12.000000,6,-1,32.857143,-37.309973,11.571429
8,1400,80,15,14,11.000000,13.000000,31,5.000000,8,10.000000,...,18,15.000000,14,7,14.000000,16,5,13.142857,-21.142857,12.714286
9,1400,80,20,26,40.000000,32.000000,29,32.000000,13,8.000000,...,17,21.000000,18,24,13.000000,4,-5,25.714286,17.142857,13.142857


In [6]:
def calRelTable(df_features, df_y, col_y):
    # col_y = "AA5052_L2"
    df_relevance = calculate_relevance_table(
        df_features, df_y[col_y], ml_task="regression"
    )
    df_relevance["y"] = col_y
    print(f"Calculated relevance for {col_y}")
    return df_relevance

In [7]:
# Identify target columns
cols_y = [col for col in stresses.columns if col not in ["sample_no", "R", "W", "D"]]
print(cols_y)

['AA5052_L1', 'AA5052_L2', 'AA5052_L3', 'AA5052_L4', 'AA5052_L5', 'AA5052_L6', 'AA5052_L7', 'AA6061_L1', 'AA6061_L2', 'AA6061_L3', 'AA6061_L4', 'AA6061_L5', 'AA6061_L6', 'AA6061_L7', 'Center_L1', 'Center_L2', 'Center_L3', 'Center_L4', 'Center_L5', 'Center_L6', 'Center_L7', 'AA5052_mean', 'AA6061_mean', 'Center_mean']


In [8]:
# Calculate relevance for dwell features
data_arr = []
for col_y in cols_y:
    r = calRelTable(ext_dwell, stresses, col_y)
    r = r.reset_index(
        drop=True
    )  # Remove extra "feature" columns since it appears in both index and data columns
    data_arr.append(r)

relevances_dwell = pd.concat(data_arr)


Calculated relevance for AA5052_L1
Calculated relevance for AA5052_L2
Calculated relevance for AA5052_L3
Calculated relevance for AA5052_L4
Calculated relevance for AA5052_L5
Calculated relevance for AA5052_L6
Calculated relevance for AA5052_L7
Calculated relevance for AA6061_L1
Calculated relevance for AA6061_L2
Calculated relevance for AA6061_L3
Calculated relevance for AA6061_L4
Calculated relevance for AA6061_L5
Calculated relevance for AA6061_L6
Calculated relevance for AA6061_L7
Calculated relevance for Center_L1
Calculated relevance for Center_L2
Calculated relevance for Center_L3
Calculated relevance for Center_L4
Calculated relevance for Center_L5
Calculated relevance for Center_L6
Calculated relevance for Center_L7
Calculated relevance for AA5052_mean
Calculated relevance for AA6061_mean
Calculated relevance for Center_mean


In [9]:
# Calculate relevance for weld features
data_arr = []
for col_y in cols_y:
    r = calRelTable(ext_weld, stresses, col_y)
    r = r.reset_index(
        drop=True
    )  # Remove extra "feature" columns since it appears in both index and data columns
    data_arr.append(r)

relevances_weld = pd.concat(data_arr)

Calculated relevance for AA5052_L1
Calculated relevance for AA5052_L2
Calculated relevance for AA5052_L3
Calculated relevance for AA5052_L4
Calculated relevance for AA5052_L5
Calculated relevance for AA5052_L6
Calculated relevance for AA5052_L7
Calculated relevance for AA6061_L1
Calculated relevance for AA6061_L2
Calculated relevance for AA6061_L3
Calculated relevance for AA6061_L4
Calculated relevance for AA6061_L5
Calculated relevance for AA6061_L6
Calculated relevance for AA6061_L7
Calculated relevance for Center_L1
Calculated relevance for Center_L2
Calculated relevance for Center_L3
Calculated relevance for Center_L4
Calculated relevance for Center_L5
Calculated relevance for Center_L6
Calculated relevance for Center_L7
Calculated relevance for AA5052_mean
Calculated relevance for AA6061_mean
Calculated relevance for Center_mean


In [10]:
with pd.ExcelWriter("S03_af_feature_relevance.xlsx", engine="openpyxl") as writer:
    relevances_dwell.to_excel(writer, sheet_name="af_dwell", index=False)
    relevances_weld.to_excel(writer, sheet_name="af_weld", index=False)